In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import time
import dill
import pandas as pd
import plotly.express as ply_exp
import plotly.graph_objects as ply_go

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:
nomes = [
    "WENO-JS", "WENO-JS (MS)", "WENO-JS (BI)", "WENO-JS (BIm)",
    "WENO-Z" , "WENO-Z (MS)" , "WENO-Z (BI)" , "WENO-Z (BIm)" ,
    "WENO-Z+", "WENO-ZC (MS)", "WENO-ZC (BI)", "WENO-ZC (BIm)",
    "WENO-D" , "WENO-A"
]
cores = [
    "#303387", "#4262C1", "#7787D9", "#ABB6FF", # WENO-JS: Azul
    "#892937", "#C23C52", "#DB6579", "#FF8FAB", # WENO-Z : Vermelho
    "#5A2571", "#801AA2", "#A23CB1", "#C66ABE", # WENO-ZC: Roxo
    "#487135", "#83B35E"                        # WENO-D : Verde
] # Cores atualizadas

API      = API_Numpy
equation = transp_equation
γ        = 1.4

WENOs = {}
WENOs["WENO-Z"] = simulation(API, equation, WENO_Z_scheme, γ, mapping = null_mapping, map_function = None, ε=ε_default).Sim

Weights = {}

Weights[nomes[ 0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 1]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 2]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 3]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Get_weights

Weights[nomes[ 4]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 5]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 6]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 7]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Get_weights

Weights[nomes[ 8]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[ 9]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=Hong_mapping, network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[10]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI_mapping  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[11]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Get_weights

Weights[nomes[12]] = simulation(API, equation, WENO_D_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[13]] = simulation(API, equation, WENO_A_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Get_weights


In [3]:
Δx = 0.01

# f_test = lambda x: np.sin(4*np.pi*x)

def f_test(x):
    z = tf.constant(-0.7, dtype=dtype)
    δ = tf.constant(0.005, dtype=dtype)
    β = tf.math.log(tf.constant(2.0, dtype=dtype))/(36.0*(δ**2.0))
    a = tf.constant(0.5, dtype=dtype)
    α = tf.constant(10, dtype=dtype)
    
    def G(x, β, z):
        return tf.math.exp(-β*(x-z)**2)
    
    def F(x, α, a):
        return tf.math.sqrt(tf.math.maximum(1-(α**2)*((x-a)**2),0))
    
    f1 = (G(x, β, z-δ) + 4*G(x, β, z) + G(x, β, z+δ))/6 # x in [-0.8, -0.6]
    f2 = 1                                              # x in [-0.4, -0.2]
    f3 = 1-tf.math.abs(10*(x-0.1))                      # x in [ 0.0,  0.2]
    f4 = (F(x, α, a-δ) + 4*F(x, α, a) + F(x, α, a+δ))/6 # x in [ 0.4,  0.6]
    f5 = 0                                              # otherwise
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.8),
        tf.math.less_equal(x, -0.6)
    )
    f1 = f1*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, -0.4),
        tf.math.less_equal(x, -0.2)
    )
    f2 = f2*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.0),
        tf.math.less_equal(x, 0.2)
    )
    f3 = f3*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    condition = tf.math.logical_and(
        tf.math.greater_equal(x, 0.4),
        tf.math.less_equal(x, 0.6)
    )
    f4 = f4*tf.cast(tf.where(condition, 1.0, 0.0), dtype=dtype)
    
    f = f1 + f2 + f3 + f4 + f5
    
    return f

In [4]:
t_final  = 2
n        = 400
CFL      = 0.2
fator    = 10   # Quantas vezes menor é a malha para a solução de referência

fronteira = FronteiraPeriodica

Δx   = 2/n
x    = np.arange(-1, 1, Δx, dtype=dtype)
init = lambda t: np.concatenate([x[t:], x[:t]], axis = 0)
u    = WENOs["WENO-Z"](f_test(x), t_final, Δx, CFL, fronteira)

pesos_0 = {}
pesos_2 = {}
for nome in nomes:
#     pesos_0[nome] = Weights[nome](f_test(x), Δx, d1, fronteira)[0]
    pesos_0[nome] = Weights[nome](f_test(x), Δx, d1, fronteira)[0][59:299,:]
#     pesos_2[nome] = Weights[nome](u, Δx, d1, fronteira)[0]
    pesos_2[nome] = Weights[nome](u, Δx, d1, fronteira)[0][59:299,:]
    print(nome+"              ")

n_ref  = n*fator
Δx_ref = 2/n_ref
    
pesos = [pesos_0, pesos_2]
# func  = [f_test(np.arange(-1, 1, Δx_ref, dtype=dtype)), u]
func  = [f_test(np.arange(-0.7, 0.5, Δx_ref, dtype=dtype)), u[59:299]]

WENO-JS              
WENO-JS (MS)              
WENO-JS (BI)              
WENO-JS (BIm)              
WENO-Z              
WENO-Z (MS)              
WENO-Z (BI)              
WENO-Z (BIm)              
WENO-Z+              
WENO-ZC (MS)              
WENO-ZC (BI)              
WENO-ZC (BIm)              
WENO-D              
WENO-A              


In [5]:
# Gráfico para o método fixo e variando o peso

df    = {}
y     = {}
label = {}

# aux_x     = np.arange(-1, 1, Δx    , dtype=dtype)
# aux_x_ref = np.arange(-1, 1, Δx_ref, dtype=dtype)
aux_x     = np.arange(-0.7, 0.5, Δx    , dtype=dtype)
aux_x_ref = np.arange(-0.7, 0.5, Δx_ref, dtype=dtype)
aux_x_    = [aux_x_ref, aux_x]

aux_cores = ["#4262C1", "#C23C52", "#57933B"]

for j in np.arange(2):
    for nome in nomes:

        label_0 = np.repeat([r"$\omega_{0}$"], [len(pesos[j][nome][:,0])])
        label_1 = np.repeat([r"$\omega_{1}$"], [len(pesos[j][nome][:,1])])
        label_2 = np.repeat([r"$\omega_{2}$"], [len(pesos[j][nome][:,2])])

        x           = np.concatenate([              aux_x,               aux_x,               aux_x], axis = 0)
        y[nome]     = np.concatenate([pesos[j][nome][:,0], pesos[j][nome][:,1], pesos[j][nome][:,2]], axis = 0)
        label[nome] = np.concatenate([            label_0,             label_1,             label_2], axis = 0)

        df[nome] = pd.DataFrame(
            {
                "x"     : x     ,
                "y"     : y[nome]     ,
                "label" : label[nome]
            }
        )

        fig = ply_exp.line(
            data_frame = df[nome]    ,
            x          = x           ,
            y          = y[nome]     ,
            color      = label[nome] ,
            log_y      = True        ,
            color_discrete_sequence = aux_cores
        )

        fig.update_layout(
            paper_bgcolor = "#FFFFFF",
            plot_bgcolor  = "#FFFFFF",
            xaxis         = dict(
                linecolor = "#000000", 
                ticks = "inside"
            ),
            yaxis         = dict(
                linecolor = "#000000", 
                title     = "", 
                tickmode  = "array",
                tickvals  = [0.1, 0.3, 0.6],
                ticktext  = ["0.1", "0.3", "0.6"]
            ),
            legend = dict(
                title       = ""        ,
                yanchor     = "top"     ,
                y           = 0.99      ,
                xanchor     = "right"   ,
                x           = 0.99      ,
                bordercolor = "#666666" ,
                borderwidth = 1
            )
        )

        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x_[j]                            ,
                y    = np.exp(func[j])                      , 
                mode = "lines"                              ,
                line = ply_go.scatter.Line(color="#666666") ,
                name = "Function"                           ,
                showlegend = False
            )
        )

        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                               ,
                y    = np.repeat(0.1, len(aux_x))                          ,
                mode = "lines"                                             ,
                line = ply_go.scatter.Line(color="#666666", dash = "dash") ,
                name = "Ideal Weight 0"                                    ,
                showlegend = False
            )
        )

        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                               ,
                y    = np.repeat(0.6, len(aux_x))                          ,
                mode = "lines"                                             ,
                line = ply_go.scatter.Line(color="#666666", dash = "dash") ,
                name = "Ideal Weight 1"                                    ,
                showlegend = False
            )
        )

        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                               ,
                y    = np.repeat(0.3, len(aux_x))                          ,
                mode = "lines"                                             ,
                line = ply_go.scatter.Line(color="#666666", dash = "dash") ,
                name = "Ideal Weight 2"                                    ,
                showlegend = False
            )
        )

        fig.update_yaxes(exponentformat="power")
    #     fig.show()
        fig.write_html("htmls/pesos/formas pesos " + nome + " (T=" + str(2*j) + ").html")
        fig.write_image("htmls/pesos/formas pesos " + nome + " (T=" + str(2*j) + ").png", width = 720, height = 720)

In [6]:
#---------------------------------------------------------------------------------------------------------------------------------------------------------------
nomes = ["WENO-JS", "WENO-Z" , "WENO-Z (MS)", "WENO-Z (BIm)", "WENO-D" ]
cores = ["#4262C1", "#C23C52", "#57933B"    , "#A23CB1"     , "#C2881D"] # Cores atualizadas

API      = API_Numpy
equation = transp_equation
γ        = 1.4

WENOs = {}
WENOs["WENO-Z"] = simulation(API, equation, WENO_Z_scheme, γ, mapping = null_mapping, map_function = None, ε=ε_default).Sim

Weights = {}

Weights[nomes[0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[1]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[2]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[3]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Get_weights
Weights[nomes[4]] = simulation(API, equation, WENO_D_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Get_weights
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Gráfico para o peso fixo e variando o método

df    = {}
y     = {}
label = {}

# aux_x     = np.arange(-1, 1, Δx    , dtype=dtype)
# aux_x_ref = np.arange(-1, 1, Δx_ref, dtype=dtype)
aux_x     = np.arange(-0.7, 0.5, Δx    , dtype=dtype)
aux_x_ref = np.arange(-0.7, 0.5, Δx_ref, dtype=dtype)
aux_x_    = [aux_x_ref, aux_x]

ideal_weights = [0.1, 0.6, 0.3]

for k in np.arange(2):
    for i in np.arange(3):

        x             = aux_x
        y[str(i)]     = pesos[k][nomes[0]][:,i]
        label[str(i)] = np.repeat([nomes[0]], [len(pesos[k][nomes[0]])])

        for j in range(1, len(nomes)):
            x             = np.concatenate([            x,                                            aux_x], axis = 0)
            y[str(i)]     = np.concatenate([    y[str(i)],                          pesos[k][nomes[j]][:,i]], axis = 0)
            label[str(i)] = np.concatenate([label[str(i)], np.repeat([nomes[j]], [len(pesos[k][nomes[j]])])], axis = 0)

        df[str(i)] = pd.DataFrame(
            {
                "x"     : x     ,
                "y"     : y[str(i)]  ,
                "label" : label[str(i)]
            }
        )

        fig = ply_exp.line(
            data_frame = df[str(i)]    ,
            x          = x             ,
            y          = y[str(i)]     ,
            color      = label[str(i)] ,
            log_y      = True          ,
            color_discrete_sequence = cores
        )

        fig.update_layout(
            paper_bgcolor = "#FFFFFF",
            plot_bgcolor  = "#FFFFFF",
            xaxis         = dict(
                linecolor = "#000000", 
                ticks = "inside"
            ),
            yaxis         = dict(
                linecolor = "#000000", 
                title     = "", 
                tickmode  = "array",
                tickvals  = [0.1, 0.3, 0.6],
                ticktext  = ["0.1", "0.3", "0.6"]
            ),
            legend = dict(
                title       = ""        ,
                yanchor     = "top"     ,
                y           = 0.99      ,
                xanchor     = "right"   ,
                x           = 0.99      ,
                bordercolor = "#666666" ,
                borderwidth = 1
            )
        )

        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x_[k]                            ,
                y    = np.exp(func[k])                      , 
                mode = "lines"                              ,
                line = ply_go.scatter.Line(color="#666666") ,
                name = "Function"                           ,
                showlegend = False
            )
        )

        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                               ,
                y    = np.repeat(ideal_weights[i], len(aux_x))             ,
                mode = "lines"                                             ,
                line = ply_go.scatter.Line(color="#666666", dash = "dash") ,
                name = "Ideal Weight"                                      ,
                showlegend = False
            )
        )

        fig.update_yaxes(exponentformat="power")
    #     fig.show()
        fig.write_html("htmls/pesos/formas peso omega " + str(i) + " (T=" + str(2*k) + ").html")
        fig.write_image("htmls/pesos/formas pesos omega " + str(i) + " (T=" + str(2*k) + ").png", width = 720, height = 720)
#         fig.write_html("htmls/pesos/formas peso omega " + str(i) + " (T=" + str(2*k) + ", all methods).html")
#         fig.write_image("htmls/pesos/formas pesos omega " + str(i) + " (T=" + str(2*k) + ", all methods).png", width = 720, height = 720)